UPDATING TRAINING / DEPLY DATA

In [1]:
from tqdm import tqdm_notebook, tnrange
import pandas as pd
import numpy as np
from random import shuffle
import pandas as pd
%matplotlib inline
import os
import random
import itertools
import hickle as hkl
import yaml
import boto3
#os.environ['KMP_DUPLICATE_LIB_OK']='True'

### Update CEO SURVEY and label arrays

In [70]:
## THIS IS FOR CREATING THE PLOT FNAMES ex: 08001, 08002
import re 
import pandas as pd
import os

CSV_PATH = '../../data/ceo-plantations-train-v19.csv'
STR = '19'
dfs = []

# perform simple clean up of columns
df = pd.read_csv(CSV_PATH, encoding = "ISO-8859-1")
df.columns = [re.sub(r'\W+', '', x) for x in df.columns]
df.rename(columns={'ïplotid':'plotid'}, inplace=True)
df.columns = [x.upper() for x in df.columns]
df.columns = ['PLOT_ID' if x == 'PLOTID' else x for x in df.columns]
df.columns = ['SAMPLE_ID' if x == 'SAMPLEID' else x for x in df.columns]
# create a col called plot_fname to hold new plot ids
df['PLOT_FNAME'] = '0'

if len(df) > 0:
    plot_ids = []
    counter = 0
    for index, row in df.iterrows():
        if row['PLOT_ID'] not in plot_ids:
            plot_ids.append(row['PLOT_ID'])
            counter += 1
        df['PLOT_FNAME'][index] = f"{str(STR).zfill(2)}{str(counter).zfill(3)}"
    print("WRITING CSV")
    df.to_csv(CSV_PATH, index = False)

/var/folders/1v/8qs7l7157sddx1_9z9wqldrc0000gp/T/ipykernel_24083/2202325372.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PLOT_FNAME'][index] = f"{str(STR).zfill(2)}{str(counter).zfill(3)}"


WRITING CSV


In [4]:
print(f"{str('08').zfill(2)}{str(counter).zfill(3)}")

08225


In [2]:
## THIS ADDS A PLANTATION COLUMN MIRRORING SYSTEM
## for some reason v8 isnt cooperating
v = 'v20'
df = pd.read_csv(f'../../data/ceo-plantations-train-{v}.csv')
df['PLANTATION'] = df.SYSTEM.map({'Not plantation': 0, 
                                    'Monoculture': 1,
                                    'Agroforestry': 2,
                                    'Unknown': 255})
# df['SYSTEM'] = df.PLANTATION.map({0:'Not plantation', 
#                                   1:'Monoculture',
#                                   2: 'Agroforestry',
#                                   255:'Unknown'})
df.head()

,index,PLOT_ID,SAMPLE_ID,LON,LAT,EMAIL,FLAGGED,COLLECTION_TIME,ANALYSIS_DURATION,IMAGERY_TITLE,IMAGERY_ATTRIBUTIONS,SAMPLE_GEOM,SYSTEM,PLOT_FNAME,PLANTATION
0,0,0,-195,-0.894849,6.116158,jessica.ertel@wri.org,False,2023-06-28 19:26,98.8 secs,Mapbox Satellite,NaN,POINT(-0.894848875934758 6.116157612001956),Unknown,20001,255
1,1,0,-194,-0.894849,6.116247,jessica.ertel@wri.org,False,2023-06-28 19:26,98.8 secs,Mapbox Satellite,NaN,POINT(-0.894848875934758 6.116246932196532),Unknown,20001,255
2,2,0,-193,-0.894849,6.116336,jessica.ertel@wri.org,False,2023-06-28 19:26,98.8 secs,Mapbox Satellite,NaN,POINT(-0.894848875934758 6.116336252376186),Unknown,20001,255
3,3,0,-192,-0.894849,6.116426,jessica.ertel@wri.org,False,2023-06-28 19:26,98.8 secs,Mapbox Satellite,NaN,POINT(-0.894848875934758 6.116425572540933),Unknown,20001,255
4,4,0,-191,-0.894849,6.116515,jessica.ertel@wri.org,False,2023-06-28 19:26,98.8 secs,Mapbox Satellite,NaN,POINT(-0.894848875934758 6.116514892690745),Unknown,20001,255


In [3]:
df.to_csv(f'../../data/ceo-plantations-train-{v}.csv', index = False)

In [4]:
df

,index,PLOT_ID,SAMPLE_ID,LON,LAT,EMAIL,FLAGGED,COLLECTION_TIME,ANALYSIS_DURATION,IMAGERY_TITLE,IMAGERY_ATTRIBUTIONS,SAMPLE_GEOM,SYSTEM,PLOT_FNAME,PLANTATION
0,0,0,-195,-0.894849,6.116158,jessica.ertel@wri.org,False,2023-06-28 19:26,98.8 secs,Mapbox Satellite,NaN,POINT(-0.894848875934758 6.116157612001956),Unknown,20001,255
1,1,0,-194,-0.894849,6.116247,jessica.ertel@wri.org,False,2023-06-28 19:26,98.8 secs,Mapbox Satellite,NaN,POINT(-0.894848875934758 6.116246932196532),Unknown,20001,255
2,2,0,-193,-0.894849,6.116336,jessica.ertel@wri.org,False,2023-06-28 19:26,98.8 secs,Mapbox Satellite,NaN,POINT(-0.894848875934758 6.116336252376186),Unknown,20001,255
3,3,0,-192,-0.894849,6.116426,jessica.ertel@wri.org,False,2023-06-28 19:26,98.8 secs,Mapbox Satellite,NaN,POINT(-0.894848875934758 6.116425572540933),Unknown,20001,255
4,4,0,-191,-0.894849,6.116515,jessica.ertel@wri.org,False,2023-06-28 19:26,98.8 secs,Mapbox Satellite,NaN,POINT(-0.894848875934758 6.116514892690745),Unknown,20001,255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31551,31551,160,31356,-6.596470,5.842162,jessica.ertel@wri.org,False,2023-06-28 20:53,31.1 secs,Mapbox Satellite,NaN,POINT(-6.596469639065242 5.842162236420776),Unknown,20161,255
31552,31552,160,31357,-6.596470,5.842252,jessica.ertel@wri.org,False,2023-06-28 20:53,31.1 secs,Mapbox Satellite,NaN,POINT(-6.596469639065242 5.84225160136414),Unknown,20161,255
31553,31553,160,31358,-6.596470,5.842341,jessica.ertel@wri.org,False,2023-06-28 20:53,31.1 secs,Mapbox Satellite,NaN,POINT(-6.596469639065242 5.842340966293222),Unknown,20161,255
31554,31554,160,31359,-6.596470,5.842430,jessica.ertel@wri.org,False,2023-06-28 20:53,31.1 secs,Mapbox Satellite,NaN,POINT(-6.596469639065242 5.842430331208064),Unknown,20161,255


In [62]:
# use this to upload the clean csv and update labels
def load_csv(csv_path, output_folder):
    
    df = pd.read_csv(csv_path, encoding = "ISO-8859-1")
    keep_columns = ['PLOT_ID', 'SAMPLE_ID', 'LON', 'LAT', 'PLANTATION', 'PLOT_FNAME'] # switch'PlANTATION' to 'SYSTEM' for multiclass
    for column in df.columns:
        if column not in keep_columns:
            df = df.drop(column, axis = 1)
    df = df.dropna(axis = 0)
    plot_ids = sorted(df['PLOT_ID'].unique())
    #print(plot_ids)
    return df, plot_ids


def reconstruct_images(plot):
    '''
    Takes a plot ID and subsets the input pd.DataFrame to that plot ID
       returns a (14, 14) array-like list with binary labels
       
        Parameters:
          batch_ids (list):
          batch_size (int):
          
         Returns:
          x_batch (arr):
          y_batch (arr):
    '''

    subs = df[df['PLOT_ID'] == plot]
    rows = []
    #print(subs.head(5))

    # reversed() computes the reverse of a given sequence 
    # object and returns it in the form of a list.
    # method for getting lat column as a list?
    lats = reversed(sorted(subs['LAT'].unique()))

    for i, val in enumerate(lats):

        # filter to row
        subs_lat = subs[subs['LAT'] == val]
        subs_lat = subs_lat.sort_values('LON', axis = 0)

        if 'PLANTATION' in df.columns:
            print(list(subs_lat['PLANTATION']))
            rows.append(list(subs_lat['PLANTATION']))

        elif 'SYSTEM' in df.columns:
            for i, val in subs_lat.iterrows():
                if val['SYSTEM'] == 'Not plantation':
                    rows.append(0)
                if val['SYSTEM'] == 'Monoculture':
                    rows.append(1)
                if val['SYSTEM'] == 'Agroforestry':
                    rows.append(2)
                if val['SYSTEM'] == 'Unknown':
                    rows.append(255)
    
    return rows

## MINE
# now recreate the label arrays and save to train-labels folder

OUTPUT_FOLDER = '../../data/train-labels/'

df, plot_ids = load_csv('../../data/ceo-plantations-train-v11.csv', OUTPUT_FOLDER)
plot_fname = sorted(df['PLOT_FNAME'].unique())

for i, x in zip(plot_ids, plot_fname):
    #assert x - 8000 == i, print(x, i)
    plot = reconstruct_images(i)
    plot = np.array(plot)
    np.save(f"{OUTPUT_FOLDER}{str(x).zfill(5)}.npy", plot)


# confirm the new labels do not match the old
assert np.load(f'{OUTPUT_FOLDER}08105.npy').all() == np.load(f'{OUTPUT_FOLDER}/_old/08105.npy').all()

In [9]:
def load_label(idx, classes, directory = '../../data/train-labels/'):
    '''
    The labels are stored as a binary 14 x 14 float64 array.
    Unless they are stored as (196,) and need to be reshaped.
    Dtype needs to be converted to float32.
    
    For binary (2 class) classification, update labels by converting
    AF to 1. For 3 class classification, leave labels as is. For 
    4 class classification, use the ttc data to update labels
    for any vegetation >= 20% tree cover as natural trees.
    
    0: no tree
    1: monoculture
    2: agroforestry
    3: natural tree
    '''
    labels_raw = np.load(directory + str(idx) + '.npy')

    if len(labels_raw.shape) == 1:
        labels_raw = labels_raw.reshape(14, 14)

    if classes == 2:
        labels = labels_raw.copy()
        labels[labels_raw == 2] = 1
        labels = labels.astype(np.float32)

    if classes == 4:
        feats = hkl.load('../../data/train-features-ard/' + str(idx) + '.hkl')
        ttc = feats[..., 0] 
        labels = labels_raw.copy()
        noplant_mask = np.ma.masked_less(labels, 1)
        notree_mask = np.ma.masked_greater(ttc, .20000000)
        mask = np.logical_and(noplant_mask.mask, notree_mask.mask)
        labels[mask] = 3
        
    else:
        labels = labels_raw.astype(np.float32)
    
    return labels

In [2]:
feats = hkl.load('../../data/train-features-ard/08004.hkl')
labels = np.load('../../data/train-labels/08004.npy')
ttc = feats[..., 0]
ttc.min(), ttc.max()

(0.06333926320075989, 0.9816009998321533)

In [3]:
labels

array([[1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [4]:
ttc

array([[0.97187835, 0.95478213, 0.94391191, 0.93478274, 0.89095354,
        0.9063344 , 0.95311332, 0.91887927, 0.94166577, 0.96370047,
        0.97481078, 0.97289568, 0.97888184, 0.97378981],
       [0.95121461, 0.93024814, 0.96080196, 0.93442822, 0.89594334,
        0.88812077, 0.8225162 , 0.89818943, 0.94080222, 0.97144699,
        0.98052162, 0.97784132, 0.97680032, 0.9776054 ],
       [0.59540081, 0.63259953, 0.79196692, 0.72071588, 0.71120298,
        0.49763751, 0.36334068, 0.837345  , 0.95908868, 0.97542977,
        0.97373855, 0.97070628, 0.97051895, 0.97402179],
       [0.27821988, 0.14729938, 0.1667268 , 0.14961672, 0.1394231 ,
        0.11423373, 0.1274597 , 0.69962275, 0.94569385, 0.9722808 ,
        0.97335589, 0.96531284, 0.9682821 , 0.96836048],
       [0.26030397, 0.11038744, 0.09603161, 0.07832757, 0.0752241 ,
        0.14105538, 0.14726451, 0.54469067, 0.87035286, 0.95600659,
        0.96961218, 0.97170401, 0.95733547, 0.96542215],
       [0.16654268, 0.0908691 , 0.0